## 1. Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import yfinance as yf
from statsmodels.tsa.holtwinters import ExponentialSmoothing, Holt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


## 2. Load and Explore Gold (GLD) Data

In [2]:
# Fetch gold price data
gold = yf.download('GLD', start='2015-01-01', end='2026-01-18', progress=False)
gold['Price'] = (gold['Close'] * 10.8).round(0)  # Convert to approximate gold price per ounce
gold = gold[['Price']].copy()

print(f"Gold data shape: {gold.shape}")
print(f"Date range: {gold.index.min()} to {gold.index.max()}")
print(f"\nData summary:")
print(gold['Price'].describe())
print(f"\nMissing values: {gold['Price'].isna().sum()}")

Gold data shape: (2777, 1)
Date range: 2015-01-02 00:00:00 to 2026-01-16 00:00:00

Data summary:
count    2777.000000
mean     1800.807706
std       669.254683
min      1085.000000
25%      1306.000000
50%      1725.000000
75%      1955.000000
max      4600.000000
Name: Price, dtype: float64

Missing values: 0


## 3. Visualize Historical Prices and Identify Trend

In [3]:
# Plot historical prices
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=gold.index, y=gold['Price'],
    mode='lines',
    name='Gold Price (GLD)',
    line=dict(color='gold', width=2)
))

# Add 50-day and 200-day moving averages
fig.add_trace(go.Scatter(
    x=gold.index, y=gold['Price'].rolling(50).mean(),
    mode='lines',
    name='50-day MA',
    line=dict(color='orange', width=1, dash='dash')
))

fig.add_trace(go.Scatter(
    x=gold.index, y=gold['Price'].rolling(200).mean(),
    mode='lines',
    name='200-day MA',
    line=dict(color='darkred', width=1, dash='dash')
))

fig.update_layout(
    title='Gold (GLD) Historical Prices with Moving Averages',
    xaxis_title='Date',
    yaxis_title='Price ($)',
    hovermode='x unified',
    template='plotly_white',
    height=500
)
fig.show()

print("✓ Historical prices visualized")

✓ Historical prices visualized


## 4. Prepare Data: Train-Test Split

In [4]:
# Use differenced series for stationarity (from Day 5-6)
gold['Price_Diff'] = gold['Price'].diff()
gold_diff = gold['Price_Diff'].dropna()

# 80-20 train-test split
train_size = int(len(gold_diff) * 0.8)
train = gold_diff.iloc[:train_size]
test = gold_diff.iloc[train_size:]

print(f"Training set: {len(train)} observations ({len(train)/len(gold_diff)*100:.1f}%)")
print(f"Test set: {len(test)} observations ({len(test)/len(gold_diff)*100:.1f}%)")
print(f"\nTrain period: {train.index.min()} to {train.index.max()}")
print(f"Test period: {test.index.min()} to {test.index.max()}")

Training set: 2220 observations (80.0%)
Test set: 556 observations (20.0%)

Train period: 2015-01-05 00:00:00 to 2023-10-27 00:00:00
Test period: 2023-10-30 00:00:00 to 2026-01-16 00:00:00


## 5. Manual Holt's Implementation

Implement Holt's method from scratch to understand the mechanics

In [5]:
def manual_holts(data, alpha, beta, forecast_periods=1):
    """
    Manual Holt's Linear Trend implementation.
    
    Parameters:
    -----------
    data : array-like
        Time series data
    alpha : float
        Level smoothing parameter (0 < alpha < 1)
    beta : float
        Trend smoothing parameter (0 < beta < 1)
    forecast_periods : int
        Number of periods to forecast ahead
    
    Returns:
    --------
    fitted : array
        Fitted values
    forecasts : array
        h-step ahead forecasts
    """
    n = len(data)
    fitted = np.zeros(n)
    level = np.zeros(n)
    trend = np.zeros(n)
    
    # Initialize level and trend
    level[0] = data[0]
    trend[0] = data[1] - data[0]  # Initial trend
    fitted[0] = level[0] + trend[0]
    
    # Recursively update level and trend
    for t in range(1, n):
        # Update level
        level[t] = alpha * data[t] + (1 - alpha) * (level[t-1] + trend[t-1])
        
        # Update trend
        trend[t] = beta * (level[t] - level[t-1]) + (1 - beta) * trend[t-1]
        
        # Fitted value (1-step ahead forecast)
        fitted[t] = level[t] + trend[t]
    
    # Generate multi-step forecasts
    forecasts = np.zeros(forecast_periods)
    for h in range(1, forecast_periods + 1):
        forecasts[h-1] = level[-1] + h * trend[-1]
    
    return fitted, level, trend, forecasts

# Test manual implementation
fitted_manual, level_manual, trend_manual, _ = manual_holts(train.values, alpha=0.3, beta=0.1, forecast_periods=1)

print("Manual Holt's Implementation Demo:")
print(f"Alpha: 0.3 (level smoothing)")
print(f"Beta: 0.1 (trend smoothing)")
print(f"\nFirst 5 fitted values: {fitted_manual[:5]}")
print(f"Last 5 fitted values: {fitted_manual[-5:]}")
print(f"\nFinal level: {level_manual[-1]:.2f}")
print(f"Final trend: {trend_manual[-1]:.4f}")

Manual Holt's Implementation Demo:
Alpha: 0.3 (level smoothing)
Beta: 0.1 (trend smoothing)

First 5 fitted values: [14.        9.       -1.61     -8.2387   -6.511629]
Last 5 fitted values: [11.15301045  7.84306963  7.93081897  6.34431894 11.73343935]

Final level: 11.34
Final trend: 0.3924


## 6. Implement Holt's Using Statsmodels

In [6]:
# Fit Holt's model with optimization
model_holt = Holt(train)
fit_holt = model_holt.fit(optimized=True)

print("Holt's Linear Trend Model Summary:")
print(fit_holt.summary())

Holt's Linear Trend Model Summary:
                              Holt Model Results                              
Dep. Variable:             Price_Diff   No. Observations:                 2220
Model:                           Holt   SSE                         505032.069
Optimized:                       True   AIC                          12056.195
Trend:                       Additive   BIC                          12079.016
Seasonal:                        None   AICC                         12056.233
Seasonal Periods:                None   Date:                 Sun, 18 Jan 2026
Box-Cox:                        False   Time:                         13:15:54
Box-Cox Coeff.:                  None                                         
                       coeff                 code              optimized      
------------------------------------------------------------------------------
smoothing_level            0.1049784                alpha                 True
smoothing_trend  

## 7. Extract and Analyze Optimized Parameters

In [7]:
# Extract parameters
optimal_alpha = fit_holt.params['smoothing_level']
optimal_beta = fit_holt.params['smoothing_trend']

print(f"Optimized Parameters:")
print(f"  Alpha (level smoothing): {optimal_alpha:.4f}")
print(f"  Beta (trend smoothing): {optimal_beta:.4f}")
print(f"\nInterpretation:")
print(f"  - Level: {optimal_alpha*100:.2f}% weight on recent observation, {(1-optimal_alpha)*100:.2f}% on smoothed history")
print(f"  - Trend: {optimal_beta*100:.2f}% weight on recent trend change, {(1-optimal_beta)*100:.2f}% on smoothed trend history")

# Extract components
final_level = fit_holt.level[-1]
final_trend = fit_holt.trend[-1]
print(f"\nFinal Components:")
print(f"  Level: {final_level:.4f}")
print(f"  Trend: {final_trend:.6f}")

Optimized Parameters:
  Alpha (level smoothing): 0.1050
  Beta (trend smoothing): 0.0680

Interpretation:
  - Level: 10.50% weight on recent observation, 89.50% on smoothed history
  - Trend: 6.80% weight on recent trend change, 93.20% on smoothed trend history

Final Components:
  Level: 10.4733
  Trend: 0.539919


## 8. Alpha and Beta Parameter Tuning

In [8]:
# Grid search for optimal alpha and beta
alphas = np.arange(0.1, 1.0, 0.2)
betas = np.arange(0.01, 0.4, 0.05)

results = []

for alpha in alphas:
    for beta in betas:
        try:
            model = Holt(train)
            fit = model.fit(smoothing_level=alpha, smoothing_trend=beta, optimized=False)
            fitted_values = fit.fittedvalues
            
            mse = mean_squared_error(train[1:], fitted_values[1:])
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(train[1:], fitted_values[1:])
            
            results.append({
                'alpha': alpha,
                'beta': beta,
                'mse': mse,
                'rmse': rmse,
                'mae': mae
            })
        except:
            pass

results_df = pd.DataFrame(results)
best_idx = results_df['rmse'].idxmin()
best_result = results_df.loc[best_idx]

print(f"Grid Search Results (Top 10 by RMSE):")
print(results_df.nsmallest(10, 'rmse')[['alpha', 'beta', 'mse', 'rmse', 'mae']].to_string(index=False))
print(f"\n✓ Best Parameters:")
print(f"  Alpha: {best_result['alpha']:.2f}, Beta: {best_result['beta']:.2f}")
print(f"  RMSE: {best_result['rmse']:.4f}")

Grid Search Results (Top 10 by RMSE):
 alpha  beta        mse      rmse       mae
   0.1  0.06 227.877893 15.095625 10.728480
   0.1  0.11 229.264629 15.141487 10.766341
   0.1  0.16 233.911150 15.294154 10.886993
   0.1  0.21 239.530714 15.476780 11.030806
   0.1  0.26 245.102941 15.655764 11.178088
   0.3  0.01 245.189072 15.658514 11.343189
   0.3  0.06 247.189421 15.722259 11.353280
   0.1  0.31 250.059517 15.813270 11.303965
   0.3  0.11 254.049424 15.938928 11.525931
   0.1  0.36 254.545032 15.954467 11.415382

✓ Best Parameters:
  Alpha: 0.10, Beta: 0.06
  RMSE: 15.0956


## 9. Generate Forecasts

In [10]:
# Generate forecasts for test period
forecast_holt = fit_holt.forecast(steps=len(test))

print(f"Forecast horizon: {len(test)} steps")
print(f"Forecast period: {test.index[0]} to {test.index[-1]}")
print(f"\nFirst 5 forecasts: {forecast_holt[:5]}")
print(f"Last 5 forecasts: {forecast_holt[-5:]}")

Forecast horizon: 556 steps
Forecast period: 2023-10-30 00:00:00 to 2026-01-16 00:00:00

First 5 forecasts: 2220    11.013197
2221    11.553116
2222    12.093034
2223    12.632953
2224    13.172871
dtype: float64
Last 5 forecasts: 2771    308.508355
2772    309.048274
2773    309.588193
2774    310.128111
2775    310.668030
dtype: float64


## 10. Evaluate Model Performance

In [11]:
# Calculate performance metrics
forecast_mse = mean_squared_error(test, forecast_holt)
forecast_rmse = np.sqrt(forecast_mse)
forecast_mae = mean_absolute_error(test, forecast_holt)
mape = np.mean(np.abs((test - forecast_holt) / test)) * 100

# Compare to naive baseline
naive_forecast = np.full(len(test), train.iloc[-1])
naive_rmse = np.sqrt(mean_squared_error(test, naive_forecast))

improvement = ((naive_rmse - forecast_rmse) / naive_rmse * 100)

print(f"Forecast Performance Metrics:")
print(f"  MSE: {forecast_mse:.4f}")
print(f"  RMSE: {forecast_rmse:.4f}")
print(f"  MAE: {forecast_mae:.4f}")
print(f"  MAPE: {mape:.2f}%")
print(f"\nBaseline Comparison:")
print(f"  Naive RMSE: {naive_rmse:.4f}")
print(f"  Improvement: {improvement:.2f}%")

if improvement > 0:
    print(f"\n✓ Holt's performs {improvement:.1f}% better than naive forecast")
else:
    print(f"\n⚠ Holt's performs {abs(improvement):.1f}% worse than naive forecast")

Forecast Performance Metrics:
  MSE: 32735.4693
  RMSE: 180.9295
  MAE: 156.4139
  MAPE: nan%

Baseline Comparison:
  Naive RMSE: 39.8656
  Improvement: -353.85%

⚠ Holt's performs 353.8% worse than naive forecast


## 11. Visualize Forecasts vs Actual Data

In [15]:
# Create forecast comparison plot
fig = go.Figure()

# Training data
fig.add_trace(go.Scatter(
    x=train.index, y=train.values,
    mode='lines',
    name='Training Data',
    line=dict(color='blue', width=1)
))

# Test data (actual)
fig.add_trace(go.Scatter(
    x=test.index, y=test.values,
    mode='lines',
    name='Test Data (Actual)',
    line=dict(color='green', width=2)
))

# Forecasts
fig.add_trace(go.Scatter(
    x=test.index, y=forecast_holt.values,
    mode='lines',
    name="Holt's Forecast",
    line=dict(color='red', width=2, dash='dash')
))

fig.update_layout(
    title="Holt's Linear Trend: Forecasts vs Actual Data",
    xaxis_title='Date',
    yaxis_title='Differenced Price',
    hovermode='x unified',
    template='plotly_white',
    height=500
)
fig.show()

print("✓ Forecast visualization complete")

✓ Forecast visualization complete


## 12. Residual Analysis and Diagnostics

In [16]:
# Calculate residuals
residuals = test - forecast_holt

print(f"Residual Statistics:")
print(f"  Mean: {residuals.mean():.4f} (should be ~0)")
print(f"  Std Dev: {residuals.std():.4f}")
print(f"  Min: {residuals.min():.4f}")
print(f"  Max: {residuals.max():.4f}")
print(f"  Q1: {residuals.quantile(0.25):.4f}")
print(f"  Q3: {residuals.quantile(0.75):.4f}")

# Create residual diagnostics plot
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Residuals Over Time', 'Residual Distribution', 'Q-Q Plot', 'ACF'),
    specs=[[{'secondary_y': False}, {'secondary_y': False}],
           [{'secondary_y': False}, {'secondary_y': False}]]
)

# Residuals time series
fig.add_trace(
    go.Scatter(x=test.index, y=residuals, mode='lines+markers', name='Residuals'),
    row=1, col=1
)
fig.add_hline(y=0, line_dash='dash', line_color='red', row=1, col=1)

# Histogram
fig.add_trace(
    go.Histogram(x=residuals, name='Distribution', nbinsx=20),
    row=1, col=2
)

fig.update_layout(height=700, showlegend=True, title_text="Residual Diagnostics")
fig.show()

print("\n✓ Residual analysis complete")

Residual Statistics:
  Mean: nan (should be ~0)
  Std Dev: nan
  Min: nan
  Max: nan
  Q1: nan
  Q3: nan



✓ Residual analysis complete


## 13. Visualize Level and Trend Components

In [17]:
# Extract level and trend components
level_values = fit_holt.level
trend_values = fit_holt.trend

fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=('Level Component', 'Trend Component', 'Actual vs Fitted'),
    specs=[[{'secondary_y': False}], [{'secondary_y': False}], [{'secondary_y': False}]],
    vertical_spacing=0.1
)

# Level
fig.add_trace(
    go.Scatter(x=train.index, y=level_values, mode='lines', name='Level', line=dict(color='blue')),
    row=1, col=1
)

# Trend
fig.add_trace(
    go.Scatter(x=train.index, y=trend_values, mode='lines', name='Trend', line=dict(color='red')),
    row=2, col=1
)
fig.add_hline(y=0, line_dash='dash', line_color='gray', row=2, col=1)

# Actual vs Fitted
fig.add_trace(
    go.Scatter(x=train.index, y=train.values, mode='lines', name='Actual', line=dict(color='blue')),
    row=3, col=1
)
fig.add_trace(
    go.Scatter(x=train.index, y=fit_holt.fittedvalues, mode='lines', name='Fitted', line=dict(color='red', dash='dash')),
    row=3, col=1
)

fig.update_xaxes(title_text='Date', row=3, col=1)
fig.update_yaxes(title_text='Level', row=1, col=1)
fig.update_yaxes(title_text='Trend', row=2, col=1)
fig.update_yaxes(title_text='Price', row=3, col=1)

fig.update_layout(height=800, showlegend=True, title_text='Holt\'s Components Visualization')
fig.show()

print("✓ Components visualization complete")

✓ Components visualization complete


## 14. Alpha-Beta Parameter Sensitivity Analysis

In [18]:
# Test different alpha-beta combinations
test_combinations = [
    {'alpha': 0.1, 'beta': 0.01, 'label': 'Low α, Low β'},
    {'alpha': 0.3, 'beta': 0.05, 'label': 'Medium α, Medium β'},
    {'alpha': 0.5, 'beta': 0.1, 'label': 'High α, Low β'},
    {'alpha': 0.3, 'beta': 0.2, 'label': 'Medium α, High β'},
    {'alpha': 0.7, 'beta': 0.3, 'label': 'High α, High β'},
]

comparison_results = []

for combo in test_combinations:
    model = Holt(train)
    fit = model.fit(smoothing_level=combo['alpha'], smoothing_trend=combo['beta'], optimized=False)
    fc = fit.forecast(steps=len(test))
    
    rmse = np.sqrt(mean_squared_error(test, fc))
    mae = mean_absolute_error(test, fc)
    
    comparison_results.append({
        'Configuration': combo['label'],
        'α': combo['alpha'],
        'β': combo['beta'],
        'RMSE': rmse,
        'MAE': mae
    })

comparison_df = pd.DataFrame(comparison_results)
print("Alpha-Beta Configuration Comparison:")
print(comparison_df.to_string(index=False))

Alpha-Beta Configuration Comparison:
     Configuration   α    β        RMSE         MAE
      Low α, Low β 0.1 0.01   43.972153   31.829709
Medium α, Medium β 0.3 0.05  133.550929  114.477668
     High α, Low β 0.5 0.10  222.652141  193.708941
  Medium α, High β 0.3 0.20   76.773830   60.784549
    High α, High β 0.7 0.30 1150.631012 1000.579614


## 15. Key Insights and Summary

In [19]:
print("="*70)
print("KEY INSIGHTS: HOLT'S LINEAR TREND METHOD")
print("="*70)

print(f"\n✓ Optimal Parameters (statsmodels):")
print(f"  α (level smoothing): {optimal_alpha:.4f}")
print(f"  β (trend smoothing): {optimal_beta:.4f}")

print(f"\n✓ Key Difference from SES:")
print(f"  SES: Flat forecasts (no trend)")
print(f"  Holt's: Trending forecasts (captures slope)")

print(f"\n✓ Forecast Interpretation:")
print(f"  Final level: {final_level:.4f}")
print(f"  Final trend: {final_trend:.6f}")
print(f"  Formula: ŷ(t+h) = {final_level:.4f} + h × {final_trend:.6f}")

print(f"\n✓ Performance vs Baseline:")
print(f"  Holt's RMSE: {forecast_rmse:.4f}")
print(f"  Naive RMSE: {naive_rmse:.4f}")
print(f"  Improvement: {improvement:.2f}%")

print(f"\n✓ When to Use Holt's:")
print(f"  ✓ Data has clear trend")
print(f"  ✓ No seasonality present")
print(f"  ✓ Need trending forecasts")
print(f"  ✗ Data is stationary (use SES instead)")
print(f"  ✗ Data has seasonality (use Holt-Winters)")

print(f"\n✓ Connection to Previous Days:")
print(f"  Day 9 (SES): Simple smoothing, no trend")
print(f"  Day 10 (Holt's): Add trend component ← YOU ARE HERE")
print(f"  Day 11 (Holt-Winters): Add seasonal component")

print("\n" + "="*70)

KEY INSIGHTS: HOLT'S LINEAR TREND METHOD

✓ Optimal Parameters (statsmodels):
  α (level smoothing): 0.1050
  β (trend smoothing): 0.0680

✓ Key Difference from SES:
  SES: Flat forecasts (no trend)
  Holt's: Trending forecasts (captures slope)

✓ Forecast Interpretation:
  Final level: 10.4733
  Final trend: 0.539919
  Formula: ŷ(t+h) = 10.4733 + h × 0.539919

✓ Performance vs Baseline:
  Holt's RMSE: 180.9295
  Naive RMSE: 39.8656
  Improvement: -353.85%

✓ When to Use Holt's:
  ✓ Data has clear trend
  ✓ No seasonality present
  ✓ Need trending forecasts
  ✗ Data is stationary (use SES instead)
  ✗ Data has seasonality (use Holt-Winters)

✓ Connection to Previous Days:
  Day 9 (SES): Simple smoothing, no trend
  Day 10 (Holt's): Add trend component ← YOU ARE HERE
  Day 11 (Holt-Winters): Add seasonal component

